# **Algorithm Trading**

This project is to demonstrate a trading strategy based on the algorithm combining the market capitalization, quantitative momentum, and P/E ratio. The index presented in this project is S&P 500 Index Fund. 

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this project is to create a portfolio model based on how much the investment funds from the client. The model will make a suggestion on how many shares of each S&P 500 constituent the client should purchase to get an equal-weight version of the index fund. I will divide the whole models in three sections: the market capitalization, return-of-period, and P/E ratio. Then I will select the optimal 50 stocks in each section

For simplicity, I built this model based on equal-weight strategy. The equal-weight strategy means the investment funds will be allocated equally on the selected stocks. Also, the data is based on free service from Wikipedia and IEX Cloud API. IEX Cloud is a platform that makes financial data and services accessible to everyone. The data is not real-time and may not be accurate. 


In [1]:
!pip install XlsxWriter

In [2]:
# import libraries
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from statistics import mean
from scipy.stats import percentileofscore as score

The list from the S&P 500 Index Fund is from Wikipedia.

In [3]:
# There are 2 tables on the Wikipedia page
# we want the first table

spy500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = spy500[0]
second_table = spy500[1]
spy_df = first_table

In [4]:
spy_df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [5]:
# Create the S&P500 into a Dataframe
stocks = spy_df['Symbol'].to_frame()
stocks.rename(columns = {'Symbol':'Ticker'}, inplace = True)
print(stocks[:10])

  Ticker
0    MMM
1    ABT
2   ABBV
3   ABMD
4    ACN
5   ATVI
6   ADBE
7    AMD
8    AAP
9    AES


In [6]:
type(stocks)

pandas.core.frame.DataFrame

To use any IEX Cloud API, you must pass an API token with each request. I upload the the token from the local drive. 

In [7]:
from google.colab import files
apiKey = files.upload()

Saving apiKey.py to apiKey (6).py


In [8]:
from apiKey import IEX_CLOUD_API_TOKEN

## **Market Capitalization**
In this section, I will create a dataframe based on the market capitalization and calculate the number of shares to buy. 

In [9]:
col_marketCap = ['Ticker', 'Price', 'Market Capitalization', 'Number of Shares to Buy']
df_marketCap = pd.DataFrame(columns = col_marketCap)

**Batch API Calls**

HTTP requests are one of the slowest components of a script. We can improve the performance by batch API calls. IEX Cloud limits their batch API calls to 100 tickers per request. Therefore, I will split the list of stocks into batch of 100 and then make a batch API call for each batch.

In [10]:
# create chunk generator
def chunks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i:i+n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))

Create the dateframe by pulling the related data from IEX Cloud.

In [12]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df_marketCap = df_marketCap.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = col_marketCap), 
                                        ignore_index = True)

In [13]:
df_marketCap

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,MMM,208.05,116944441540,N/A
1,ABT,113.18,202899274457,N/A
2,ABBV,121.15,206110244610,N/A
3,ABMD,306.94,13934147100,N/A
4,ACN,288.58,198175757159,N/A
...,...,...,...,...
500,YUM,122.37,35745153292,N/A
501,ZBRA,519.70,28054784052,N/A
502,ZBH,161.86,33362085111,N/A
503,ZION,55.17,9356115201,N/A


**Investment Fund**

This section will require the input data from the investment fund from the client in integer format. If the input data is not in integer, the model will return with an error message. 

In [14]:
portfolio_amt = input('Enter the value of your portfolio: ')
try: 
  val = float(portfolio_amt)
  print(val)
except ValueError:
  print('That is not an interger. Please try again.')
  portfolio_amt = input('Please Enter and integer')
  val = float(portfolio_amt)

Enter the value of your portfolio: 300000
300000.0


This section is to calculte thecapital on each stocks based on equal-weight strategy. The model will select the 50 stocks with the highest market capitalization. From there and will calculate the recommended trades for an equal-weight portfolio of these 50 stocks.

In [15]:
df_marketCap50 = df_marketCap.sort_values('Market Capitalization', ascending = False)
df_marketCap50 = df_marketCap[:50]
df_marketCap50 = df_marketCap50.reset_index(drop = True)

# ***The recommended trades of 50 stocks based on market capitalization***

In [17]:
portfolio_marketCap50 = val/len(df_marketCap50.index)
for i in range(0, len(df_marketCap50['Ticker'])):
  df_marketCap50.loc[i, 'Number of Shares to Buy'] = math.floor(portfolio_marketCap50/ df_marketCap50['Price'][i])

In [18]:
df_marketCap50

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,MMM,208.05,116944441540,28
1,ABT,113.18,202899274457,53
2,ABBV,121.15,206110244610,49
3,ABMD,306.94,13934147100,19
4,ACN,288.58,198175757159,20
5,ATVI,99.64,75301413372,60
6,ADBE,575.50,270766436207,10
7,AMD,82.81,98551932535,72
8,AAP,204.56,13671432903,29
9,AES,26.34,17766480104,227


Export to Excel format

In [19]:
writer= pd.ExcelWriter('Recommended Trades_Market Cap.xlsx', engine = 'xlsxwriter')
df_marketCap50.to_excel(writer, 'Recommended Trades_Market Cap', index = False)
writer.save()

# **Quantitative Momentum Strategy**

"Momentum investing" means investing in the stocks that have increased in price the most.

The investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The investors prefer the high-quality momentum stocks because the high-quality stocks yield stable positive returns. On the other hand, the low-quality momentum can often be cause by short-term news that is unpredictable or is unlikely to be repeated in the future, such as the surge of the pharmaceutical companies benefitting from the covid vaccines.

To identify high-quality momentum, we're going to build a strategy that selects the top 50 stocks from the highest percentiles of:

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

In [20]:
stable_col = [
              'Ticker', 
              'Price', 
              'Number of Shares to Buy', 
              'One-Year Price Return', 
              'One-Year Return Percentile',
              'Six-Month Price Return',
              'Six-Month Return Percentile',
              'Three-Month Price Return',
              'Three-Month Return Percentile',
              'One-Month Price Return',
              'One-Month Return Percentile',
              'Stable Return Score'
              ]

df_stable = pd.DataFrame(columns = stable_col)

In [21]:
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df_stable = df_stable.append(
                                      pd.Series([symbol, 
                                                  data[symbol]['quote']['latestPrice'],
                                                  'N/A',
                                                  data[symbol]['stats']['year1ChangePercent'],
                                                  'N/A',
                                                  data[symbol]['stats']['month6ChangePercent'],
                                                  'N/A',
                                                  data[symbol]['stats']['month3ChangePercent'],
                                                  'N/A',
                                                  data[symbol]['stats']['month1ChangePercent'],
                                                  'N/A',
                                                  'N/A'
                                                  ], 
                                                  index = stable_col), 
                                        ignore_index = True)

In [22]:
# create the list for the loop
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

In [23]:
for row in df_stable.index:
  for time_period in time_periods:
    change_col = f'{time_period} Price Return'
    percentile_col = f'{time_period} Return Percentile'
    df_stable.loc[row, f'{time_period} Return Percentile'] = score(df_stable[change_col], df_stable.loc[row, f'{time_period} Price Return'])/100

Calculating the high-quality momentum(HQM) Score
This model is to calculate the HQM Score, which is the high-quality momentum score that I'll use to filter for stocks in this investing strategy. The HQM Score will be the arithmetic mean of the 4 momentum percentile scores.

In [39]:
for row in df_stable.index:
  momentum_percentiles = []
  for time_period in time_periods:
    momentum_percentiles.append(df_stable.loc[row, f'{time_period} Return Percentile'])
    df_stable.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [40]:
df_stable50 = df_stable[:50].sort_values('HQM Score', ascending = False)
df_stable50 = df_stable50.reset_index(drop = True)

# **The recommended trades of 50 stocks with the highest momentum**

In [41]:
portfolio_stable50 = val/len(df_stable50.index)
for i in range(0, len(df_stable50['Ticker'])):
  df_stable50.loc[i, 'Number of Shares to Buy'] = math.floor(portfolio_marketCap50/ df_stable50['Price'][i])

In [42]:
df_stable50

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Stable Return Score,HQM Return Score,HQM Score
0,AMAT,140.01,42,1.492822,0.968317,0.589748,0.962376,0.222553,0.916832,0.120848,0.952475,0.95,0.950000,0.950000
1,GOOG,2635.65,2,0.790965,0.788119,0.437326,0.873267,0.234822,0.926733,0.094398,0.889109,0.869307,0.869307,0.869307
2,ANET,375.79,15,0.661799,0.69703,0.329989,0.746535,0.306553,0.984158,0.118863,0.946535,0.843564,0.843564,0.843564
3,GOOGL,2533.87,2,0.748003,0.764356,0.414882,0.849505,0.198344,0.885149,0.078388,0.851485,0.837624,0.837624,0.837624
4,A,148.57,40,0.690736,0.724752,0.252024,0.59604,0.197061,0.881188,0.103957,0.920792,0.780693,0.780693,0.780693
5,ALGN,607.52,9,1.455749,0.966337,0.202623,0.477228,0.130526,0.724752,0.066265,0.823762,0.74802,0.748020,0.748020
6,APTV,159.00,37,1.086045,0.920792,0.303359,0.70099,0.044849,0.346535,0.135618,0.966337,0.733663,0.733663,0.733663
7,AXP,173.62,34,0.666448,0.710891,0.412462,0.843564,0.107890,0.615842,0.043898,0.724752,0.723762,0.723762,0.723762
8,ALXN,185.10,32,0.663483,0.69901,0.166871,0.39802,0.207415,0.893069,0.048738,0.754455,0.686139,0.686139,0.686139
9,ALB,166.16,36,1.244909,0.950495,0.257636,0.607921,0.073541,0.477228,0.031380,0.663366,0.674752,0.674752,0.674752


Export to Excel format

In [28]:
writer= pd.ExcelWriter('Recommended Trades_HQM Score.xlsx', engine = 'xlsxwriter')
df_stable50.to_excel(writer, 'Recommended Trades_HQM Score', index = False)
writer.save()

# **Quantitative Value Strategy**
"Value investing" in this model means the stocks with the highest P/E ratio. There are many different interpretations of the best value by different ranges of the P/E ratio. For simplicity, I will just use the highest P/E ratio.

In [43]:
col_pe = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
df_pe = pd.DataFrame(columns = col_pe)

In [30]:
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df_pe = df_pe.append(
                              pd.Series([symbol, 
                                          data[symbol]['quote']['latestPrice'],
                                          data[symbol]['quote']['peRatio'],
                                          'N/A'
                                          ], 
                                        index = col_pe), 
                              ignore_index = True)

In [31]:
df_pe = df_pe[df_pe['Price-to-Earnings Ratio'] > 0]
df_pe50 = df_pe[:50].sort_values('Price-to-Earnings Ratio', ascending = False).reset_index(drop = True)

# **The recommended trades of 50 stocks based on the P/E ratios**




In [32]:
portfolio_pe50 = val/len(df_pe50.index)
for i in range(0, len(df_pe50['Ticker'])):
  df_pe50.loc[i, 'Number of Shares to Buy'] = math.floor(portfolio_pe50/ df_pe50['Price'][i])

In [33]:
df_pe50

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APA,23.43,1084.2,256
1,ALGN,626.14,108.27,9
2,APTV,161.00,92.78,37
3,AMZN,3413.55,67.15,1
4,ANSS,350.38,66.44,17
5,AMT,281.79,65.2,21
6,ABMD,306.73,64.02,19
7,ALXN,184.90,61.74,32
8,ALB,166.65,50.63,36
9,A,151.64,50.53,39


Export to Excel format

In [ ]:
writer= pd.ExcelWriter('Recommended Trades_HQM Score.xlsx', engine = 'xlsxwriter')
df_stable50.to_excel(writer, 'Recommended Trades_HQM Score', index = False)
writer.save()

Let's find out the stocks which exists on all three dataframe, i.e. the stocks "superstar" stocks.

In [48]:
top_list = list(set(df_marketCap50.Ticker) & set(df_stable50.Ticker) & set(df_pe50.Ticker))


In [47]:
print('There are', len(top_list), 'stocks exist on all three  dataframes.')
print()
print(top_list)

There are 42 stocks exist on all three  dataframes.

['ANSS', 'LNT', 'ALL', 'ALB', 'AEP', 'AFL', 'ATVI', 'ABBV', 'ANET', 'APA', 'AOS', 'AAPL', 'ADM', 'APD', 'ACN', 'APH', 'A', 'AMGN', 'GOOGL', 'AEE', 'APTV', 'AMZN', 'ALGN', 'ABT', 'ADI', 'AAP', 'ADBE', 'ALXN', 'AMT', 'AKAM', 'ABMD', 'ARE', 'ALLE', 'MO', 'AWK', 'ANTM', 'AMAT', 'AXP', 'AME', 'AMCR', 'AMD', 'MMM']


In [36]:
df_top = df_pe50[(df_pe50['Ticker'].isin(top_list))].drop(columns=['Price-to-Earnings Ratio']).reset_index(drop = True)
df_top['Number of Shares to Buy'] = np.nan

,Ticker,Price,Number of Shares to Buy
0,APA,23.43,NaN
1,ALGN,626.14,NaN
2,APTV,161.00,NaN
3,AMZN,3413.55,NaN
4,ANSS,350.38,NaN
5,AMT,281.79,NaN
6,ABMD,306.73,NaN
7,ALXN,184.90,NaN
8,ALB,166.65,NaN
9,A,151.64,NaN


# **The Superstar stocks based on all three criteria.**

In [37]:
portfolio_top = val/len(top_list)
for i in range(0, len(top_list)):
  df_top.loc[i, 'Number of Shares to Buy'] = math.floor(portfolio_top/ df_top['Price'][i])
df_top

,Ticker,Price,Number of Shares to Buy
0,APA,23.43,304.0
1,ALGN,626.14,11.0
2,APTV,161.00,44.0
3,AMZN,3413.55,2.0
4,ANSS,350.38,20.0
5,AMT,281.79,25.0
6,ABMD,306.73,23.0
7,ALXN,184.90,38.0
8,ALB,166.65,42.0
9,A,151.64,47.0


Export to Excel format

In [49]:
writer= pd.ExcelWriter('The Superstar Stocks.xlsx', engine = 'xlsxwriter')
df_stable50.to_excel(writer, 'The SuperStar Stocks', index = False)
writer.save()